In [1]:
import os
import sys
import csv
import re
import time
import base64
import numpy as np
import pandas as pd
import requests
import folium
import polyline
import geopy.distance
import geopandas as gpd
from termcolor import colored
from sklearn.cluster import DBSCAN
from scipy.spatial import distance_matrix
from scipy.optimize import minimize
from sklearn.neighbors import KDTree
from tsp_solver.greedy import solve_tsp
import matplotlib.pyplot as plt
from itertools import combinations
from geopy.distance import geodesic
from folium.plugins import MarkerCluster
from folium.plugins import BeautifyIcon
from folium.map import FeatureGroup
from cachetools import cached, LRUCache
from cachetools.keys import hashkey
from functools import partial
from concurrent.futures import ThreadPoolExecutor
import joblib

In [ ]:
#### Validate locations from locations.csv

In [ ]:
# Set the region
region = 'ssudan'

# Set the path to for region's files
locations_csv_path = 'ssudan'

# Set the file name of the locations.csv file
locations_csv = f'{region}-locations.csv'

# Check if the CSV file exists
locations_csv_file = os.path.join(locations_csv_path, locations_csv)
if not os.path.exists(locations_csv_file):
    print(f"Error: {locations_csv} file not found.")
    exit(1)

# Check if the CSV file exists
if not os.path.exists(locations_csv_file):
    print(f"Error: {locations_csv} file not found.")
    exit(1)

# Read the locations from CSV
df = pd.read_csv(locations_csv_file)

# List to keep track of invalid locations
invalid_locations = []

# Base URL for the OSRM nearest service
# osrm_endpoint = 'http://localhost:5000/nearest/v1/driving/'
osrm_endpoint = 'http://localhost:5000/nearest/v1/driving/'

# Iterate through each location in the DataFrame
for index, row in df.iterrows():
    lat = row['latitude']
    lon = row['longitude']
    name = row['name']

    # Prepare API request URL for the nearest service
    request_url = f"{osrm_endpoint}{lon},{lat}"

    # Send request to OSRM
    response = requests.get(request_url)
    data = response.json()

    # Check if the response is valid and contains a way (road)
    if response.status_code == 200 and data['code'] == 'Ok' and len(data['waypoints']) > 0:
        # This means the location is valid and near a known road
        print(f"Valid location found for {name}.")
    else:
        # The location is invalid or not near a known road
        print(f"Invalid location or not found: {name}.")
        invalid_locations.append(name)

# Optional: Handle invalid locations
if invalid_locations:
    print("\nInvalid locations or locations not found:")
    for name in invalid_locations:
        print(f" - {name}")
else:
    print("\nAll locations are valid.")

In [ ]:
#### Store visited location pairs pruning

In [ ]:
# Set the region 
region = 'ssudan'

# Set the path to for region's files
locations_csv_path = 'ssudan'

# Set the file name of the locations.csv file
locations_csv = f'{region}-small-locations.csv'

# Check if the CSV file exists
locations_csv_file = os.path.join(locations_csv_path, locations_csv)
if not os.path.exists(locations_csv_file):
    print(f"Error: {locations_csv} file not found.")
    exit(1)

# Read the locations from CSV
df = pd.read_csv(locations_csv_file)

# Create a map centered around the first location
start_lat = df.loc[0, 'latitude']
start_lon = df.loc[0, 'longitude']
m = folium.Map(location=[start_lat, start_lon], zoom_start=7)

# Create an empty list to store route instructions
route_instructions = []

# Create a set to store visited location pairs
visited_pairs = set()

distance_dict = {}

# Iterate through each pair of locations in the DataFrame
for i in range(len(df)):
    for j in range(i + 1, len(df)):
        # Extract location details
        name1 = df.loc[i, 'name']
        lat1 = df.loc[i, 'latitude']
        lon1 = df.loc[i, 'longitude']
        name2 = df.loc[j, 'name']
        lat2 = df.loc[j, 'latitude']
        lon2 = df.loc[j, 'longitude']

        # Check if the pair has been visited before (or its reverse pair)
        if (name1, name2) in visited_pairs or (name2, name1) in visited_pairs:
            continue

        # Prepare API request URL
        osrm_endpoint = 'http://localhost:5000/route/v1/driving'
        request_url = f"{osrm_endpoint}/{lon1},{lat1};{lon2},{lat2}?steps=true&geometries=polyline"

        try:
            # Send request to OSRM
            response = requests.get(request_url)
            response.raise_for_status()

            # Extract route data from the response
            route_data = response.json()

            # Check if response contains a route
            if 'routes' in route_data and len(route_data['routes']) > 0:
                # Extract route steps
                steps = route_data['routes'][0]['legs'][0]['steps']
                distance_sum = 0
                for step in steps:
                    # Extract step instructions
                    instruction = step['name']

                    # Extract step distance and add to the distance sum
                    distance = step['distance']
                    distance_sum += distance

                # Append step information to the route_instructions list
                route_instructions.append((instruction, distance_sum))
                
                # Store distance between the pair in the dictionary
                distance_dict[(name1, name2)] = round(distance_sum, 2)

                # Extract route geometry
                geometry = route_data['routes'][0]['geometry']

                # Decode polyline string to get coordinates
                coordinates = polyline.decode(geometry)

                # Add the coordinates to the map as a PolyLine
                folium.PolyLine(
                    locations=coordinates,
                    color='blue',
                    weight=2,
                    opacity=0.9
                ).add_to(m)

                # Add the pair to the visited set and its reverse pair
                visited_pairs.add((name1, name2))
                visited_pairs.add((name2, name1))

        except requests.exceptions.RequestException as e:
            print(f"Error occurred during OSRM API request: {e}")
            continue

        # Add markers for the start and end locations
        folium.CircleMarker(
            location=(lat1, lon1),
            radius=6,
            color='red',
            fill=True,
            fill_color='blue',
            popup=name1
        ).add_to(m)

        folium.CircleMarker(
            location=(lat2, lon2),
            radius=6,
            color='red',
            fill=True,
            fill_color='blue',
            popup=name2
        ).add_to(m)

# Save the map as an HTML file
map_file = os.path.join(f'{locations_csv_path}/{region}-visited-pairs-map.html')
m.save(map_file)
print(f"Map saved as {map_file}")

# Save distances to a CSV file
with open(f'{locations_csv_path}/{region}-visited-pairs-routes.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['name1', 'name2', 'distance'])
    for pair, distance in distance_dict.items():
        writer.writerow([pair[0], pair[1], distance])

print(f"Distances saved as {locations_csv_path}/{region}-visited-pairs-routes.csv")

# Display the map
m

In [3]:
#### Get hospitals along the routes

'/home/mghorbani/workspace/osrm-tutorial'

In [ ]:
import subprocess
import xml.etree.ElementTree as ET

def run_overpass_query(query):
    exec_dir = "/home/mghorbani/workspace/overpass/bin"
    db_dir = "/home/mghorbani/workspace/overpass/db"
    command = [f"{exec_dir}/osm3s_query", "--db-dir=" + db_dir]
    
    # Start the subprocess
    process = subprocess.Popen(
        command,
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )
    
    # Send the query and retrieve the results
    try:
        output, errors = process.communicate(input=query, timeout=30)  # Adjust timeout as necessary
        if process.returncode != 0:
            print(f"Error in Overpass query: {errors}")
            return None
        return output
    except subprocess.TimeoutExpired:
        process.kill()
        print("Query timed out")
        return None

def parse_hospitals(xml_data):
    root = ET.fromstring(xml_data)
    hospitals = []
    for node in root.findall('node'):
        hospital_data = {
            'id': node.get('id'),
            'lat': node.get('lat'),
            'lon': node.get('lon'),
            'name': None,
            'operator': None
        }
        for tag in node.findall('tag'):
            if tag.get('k') == 'name':
                hospital_data['name'] = tag.get('v')
            elif tag.get('k') == 'operator':
                hospital_data['operator'] = tag.get('v')
        hospitals.append(hospital_data)
    return hospitals

# Define your Overpass query
query = """
<osm-script output="xml">
  <query type="node">
    <bbox-query n="4.96" s="4.81" w="31.47" e="31.66"/>
    <has-kv k="amenity" v="hospital"/>
  </query>
  <print/>
</osm-script>
"""

# Run the query
xml_result = run_overpass_query(query)
if xml_result:
    hospitals = parse_hospitals(xml_result)
    # Process or display the results
    for hospital in hospitals:
        print(hospital)

In [ ]:
### Create location graph

In [ ]:
import base64
import os
import re
import sys
import csv
import json
import time
import joblib
import pandas as pd
import requests
import folium
import geopy.distance
import subprocess
from geopy.distance import geodesic
from folium.map import FeatureGroup
from concurrent.futures import ThreadPoolExecutor
import xml.etree.ElementTree as ET


def add_extra_text(map_object):
    # Sample locations with names
    locations = [
        {'name': 'South Sudan', 'coordinates': [7.5, 32]},
        {'name': 'CAR', 'coordinates': [6.00, 24]},
        {'name': 'Ethiopia', 'coordinates': [8, 36]},
        {'name': 'Kenya', 'coordinates': [3, 35]},
        {'name': 'Sudan', 'coordinates': [15, 29]},
        {'name': 'Uganda', 'coordinates': [1, 32]},
        {'name': 'CRC', 'coordinates': [3, 29]},
    ]

    # Add markers with text labels
    for loc in locations:
        icon = folium.DivIcon(html=f"""
            <div style="font-size:12pt; color: black;">{loc['name']}</div>
            """)
        folium.Marker(
            location=loc['coordinates'],
            icon=icon
        ).add_to(map_object)


def add_location_texts(map_object, locations_df):
    # Add markers with text labels
    for index, row in locations_df.iterrows():
        name = row['name']
        lat = row['latitude']
        lon = row['longitude']
        icon = folium.DivIcon(html=f"""
            <div style="font-size:10pt; color: black;">{name}</div>
            """)
        folium.Marker(
            location=(lat, lon),
            icon=icon
        ).add_to(map_object)


def add_location_markers(map_object, locations_df):
    for _, row in locations_df.iterrows():
        # Extract details
        name = row['name']
        lat = row['latitude']
        lon = row['longitude']
        location_type = row['location_type']
        color = 'green' if location_type == 'camp' else 'yellow' if location_type == 'town' else 'red'
        # Add marker to the map
        folium.CircleMarker(
            location=(lat, lon),
            radius=6,
            color=color,
            weight=2,
            opacity=.9,
            fill=True,
            fill_color=color,
            fill_opacity=.6,
            popup=name
        ).add_to(map_object)


cache_dir = 'cachedir'
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)

memory = joblib.Memory("cachedir", verbose=0)


@memory.cache
def get_nearest_road_coordinate(coordinate):
    """Get the nearest road coordinate for a given point."""
    nearest_url = f'http://localhost:5000/nearest/v1/driving/{coordinate[1]},{coordinate[0]}'
    response = requests.get(nearest_url)
    if response.status_code == 200:
        data = response.json()
        if data['waypoints']:
            return data['waypoints'][0]['location']  # Return lon, lat of nearest point
    else:
        print(f"Failed to find nearest road: {response.status_code} {response.text}")
    return coordinate  # Return original coordinate if no road found


@memory.cache
def get_route(start, end):
    """Cached function to get route geometry."""
    # Get nearest road coordinates for start and end
    start_road_coord = get_nearest_road_coordinate(start)
    end_road_coord = get_nearest_road_coordinate(end)

    # Use nearest road coordinates to get the route
    osrm_route_url = f'http://localhost:5000/route/v1/driving/{start_road_coord[0]},{start_road_coord[1]};{end_road_coord[0]},{end_road_coord[1]}?steps=true&geometries=geojson'
    response = requests.get(osrm_route_url)

    if response.status_code != 200:
        print(f"Failed to get route: {response.status_code} {response.text}")
        return None

    try:
        data = response.json()
        if 'routes' in data and data['routes']:
            return data['routes'][0]['geometry']
    except requests.exceptions.JSONDecodeError:
        print("Failed to decode JSON from response")

    return None


def calculate_total_distance(coordinates):
    total_distance = 0
    if len(coordinates) > 1:
        for i in range(1, len(coordinates)):
            total_distance += geopy.distance.geodesic(coordinates[i - 1][::-1], coordinates[i][::-1]).km
    return total_distance


def add_routes(map_object, locations_df):
    executor = ThreadPoolExecutor(max_workers=10)
    futures = []
    coordinates = []
    for i in range(len(locations_df)):
        for j in range(i + 1, len(locations_df)):
            loc1, loc2 = locations_df.iloc[i], locations_df.iloc[j]
            # Start asynchronous requests
            future = executor.submit(get_route, [loc1['latitude'], loc1['longitude']],
                                     [loc2['latitude'], loc2['longitude']])
            futures.append((future, loc1['name'], loc2['name']))

    # Collecting results
    for future, name1, name2 in futures:
        geometry = future.result()
        if geometry:
            total_distance = calculate_total_distance(geometry['coordinates'])
            polyline_obj = folium.PolyLine(
                locations=[(xy[1], xy[0]) for xy in geometry['coordinates']],
                color='darkorange',
                weight=2,
                opacity=0.9,
                popup=f"{name1} to {name2}"
            )
            polyline_obj.add_to(map_object)
            coordinates.append({'geometry': geometry, 'name1': name1, 'name2': name2, 'distance': total_distance})
    return coordinates


def add_direct_distance_routes(map_object, locations_df):
    executor = ThreadPoolExecutor(max_workers=10)
    futures = []
    coordinates = []
    processed_pairs = set()  # This will keep track of processed location pairs

    # First, initiate all requests
    for i in range(len(locations_df)):
        for j in range(i + 1, len(locations_df)):
            loc1, loc2 = locations_df.iloc[i], locations_df.iloc[j]
            location_pair = frozenset([loc1['name'], loc2['name']])

            # Skip this pair if it has already been processed
            if location_pair in processed_pairs:
                continue

            # Mark this pair as processed
            processed_pairs.add(location_pair)

            # Start asynchronous requests
            future = executor.submit(get_route, [loc1['latitude'], loc1['longitude']],
                                     [loc2['latitude'], loc2['longitude']])
            futures.append((future, loc1['name'], loc2['name']))

    # Then, process all futures
    for future, name1, name2 in futures:
        geometry = future.result()
        if geometry:
            total_distance = calculate_total_distance(geometry['coordinates'])
            folium.PolyLine(
                locations=[(xy[1], xy[0]) for xy in geometry['coordinates']],
                color='darkorange',
                weight=2,
                opacity=0.9,
                popup=f"{name1} to {name2}"
            ).add_to(map_object)
            coordinates.append({'geometry': geometry, 'name1': name1, 'name2': name2, 'distance': total_distance})

    return coordinates


def add_visit_tracking_routes(map_object, locations_df):
    executor = ThreadPoolExecutor(max_workers=10)
    futures = []
    visited_pairs = set()
    processed_routes = []  # List to store processed route details

    # First, initiate all requests
    for i in range(len(locations_df)):
        for j in range(i + 1, len(locations_df)):
            loc1, loc2 = locations_df.iloc[i], locations_df.iloc[j]
            name1, name2 = loc1['name'], loc2['name']
            location_pair = (name1, name2)

            # Check if pair has already been processed in either direction
            if location_pair in visited_pairs or (name2, name1) in visited_pairs:
                continue

            # Mark both directions as processed
            visited_pairs.add(location_pair)
            visited_pairs.add((name2, name1))

            # Start asynchronous route requests
            future = executor.submit(get_route, [loc1['latitude'], loc1['longitude']],
                                     [loc2['latitude'], loc2['longitude']])
            futures.append((future, name1, name2))

    # Then, process all futures for results
    for future, name1, name2 in futures:
        geometry = future.result()
        if geometry:
            total_distance = calculate_total_distance(geometry['coordinates'])
            folium.PolyLine(
                locations=[(xy[1], xy[0]) for xy in geometry['coordinates']],
                color='darkorange',
                weight=2,
                opacity=0.9,
                popup=f"{name1} to {name2}"
            ).add_to(map_object)

            # Store route information
            processed_routes.append({'geometry': geometry, 'name1': name1, 'name2': name2, 'distance': total_distance})

    return processed_routes


def add_sequential_pruned_routes(map_object, locations_df):
    executor = ThreadPoolExecutor(max_workers=10)
    futures = []
    processed_routes = []  # List to store processed route details

    # Prepare all futures first
    for i in range(len(locations_df)):
        loc1 = locations_df.iloc[i]
        if i == len(locations_df) - 1:
            loc2 = locations_df.iloc[0]  # Wrap around to the first location
        else:
            loc2 = locations_df.iloc[i + 1]

        name1, lat1, lon1 = loc1['name'], loc1['latitude'], loc1['longitude']
        name2, lat2, lon2 = loc2['name'], loc2['latitude'], loc2['longitude']

        # Start asynchronous requests
        future = executor.submit(get_route, [lat1, lon1], [lat2, lon2])
        futures.append((future, name1, name2))

    # Process futures as they complete
    for future, name1, name2 in futures:
        geometry = future.result()
        if geometry:
            total_distance = calculate_total_distance(geometry['coordinates'])
            folium.PolyLine(
                locations=[(xy[1], xy[0]) for xy in geometry['coordinates']],
                color='darkorange',
                weight=2,
                opacity=0.9,
                popup=f"{name1} to {name2}"
            ).add_to(map_object)

            # Store route information
            processed_routes.append({'geometry': geometry, 'name1': name1, 'name2': name2, 'distance': total_distance})

    return processed_routes


def add_triangle_pruned_routes(map_object, locations_df, num_neighbors=5):
    executor = ThreadPoolExecutor(max_workers=10)
    futures = []
    all_routes = []

    # Pre-calculate distances and sort neighbors for all locations
    for idx, loc1 in locations_df.iterrows():
        name1 = loc1['name']
        lat1 = loc1['latitude']
        lon1 = loc1['longitude']
        distances_with_names = []

        for idx2, loc2 in locations_df.iterrows():
            if loc2['name'] != name1:
                lat2 = loc2['latitude']
                lon2 = loc2['longitude']
                distance = geodesic((lat1, lon1), (lat2, lon2)).meters
                distances_with_names.append((loc2['name'], lat2, lon2, distance))

        # Sort distances and get nearest neighbors
        sorted_neighbors = sorted(distances_with_names, key=lambda x: x[3])[:num_neighbors]

        # Dispatch route calculation tasks for the nearest neighbors
        for name2, lat2, lon2, _ in sorted_neighbors:
            future = executor.submit(get_route, [lat1, lon1], [lat2, lon2])
            futures.append((future, name1, name2))

    # Collect results from futures
    for future, name1, name2 in futures:
        geometry = future.result()
        if geometry:
            total_distance = calculate_total_distance(geometry['coordinates'])
            folium.PolyLine(
                locations=[(xy[1], xy[0]) for xy in geometry['coordinates']],
                color='darkorange',
                weight=2,
                opacity=0.9,
                popup=f"{name1} to {name2}"
            ).add_to(map_object)

            all_routes.append({'geometry': geometry, 'name1': name1, 'name2': name2, 'distance': total_distance})

    return all_routes


def sample_route_points(geometry, num_samples=10):
    """Sample evenly spaced points along a route geometry."""
    if not isinstance(geometry, dict) or 'coordinates' not in geometry:
        raise ValueError("Invalid geometry format. Expected a dictionary with a 'coordinates' key.")

    points = geometry['coordinates']
    sampled_points = [points[i] for i in range(0, len(points), max(1, len(points) // num_samples))]
    return sampled_points


def run_overpass_query(query, exec_dir, db_dir):
    command = [f"{exec_dir}/osm3s_query", "--db-dir=" + db_dir]
    process = subprocess.Popen(
        command,
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )
    try:
        output, errors = process.communicate(input=query, timeout=30)
        if process.returncode != 0:
            print(f"Error in Overpass query: {errors}")
            return None
        return output
    except subprocess.TimeoutExpired:
        process.kill()
        print("Query timed out")
        return None


def parse_pois_from_xml(xml_data):
    root = ET.fromstring(xml_data)
    pois = []
    # Iterate over both nodes and ways
    for element in root.findall('./*'):
        tags = {tag.get('k'): tag.get('v') for tag in element.findall('tag')}
        if 'amenity' in tags and tags['amenity'] == 'hospital':
            # Common data extraction
            poi_data = {
                'type': tags.get('amenity', 'unknown'),
                'name': tags.get('name', 'Unknown'),
                'operator': tags.get('operator', 'Unknown')
            }
            # If it's a node, it will have latitude and longitude attributes directly
            if element.tag == 'node':
                poi_data.update({
                    'latitude': element.get('lat'),
                    'longitude': element.get('lon')
                })
            # If it's a way, we might need to compute the centroid or just take the first node's coordinates
            elif element.tag == 'way':
                # Retrieve the coordinates of the first referenced node
                # This is a simplification, consider calculating the geometric center if needed
                ref = element.find('nd').get('ref')
                ref_node = root.find(f".//node[@id='{ref}']")
                if ref_node is not None:
                    poi_data.update({
                        'latitude': ref_node.get('lat'),
                        'longitude': ref_node.get('lon')
                    })
            pois.append(poi_data)
    return pois



def find_pois_along_route(points, amenity, search_radius=0):
    exec_dir = "/home/mghorbani/workspace/overpass/bin"
    db_dir = "/home/mghorbani/workspace/overpass/db"
    all_pois = []

    for point in points:
        lat, lon = point[1], point[0]
        query = f"""
        <osm-script output="xml">
          <union>
            <query type="node">
              <around radius="{search_radius}" lat="{lat}" lon="{lon}"/>
              <has-kv k="amenity" v="{amenity}"/>
            </query>
            <query type="way">
              <around radius="{search_radius}" lat="{lat}" lon="{lon}"/>
              <has-kv k="amenity" v="{amenity}"/>
            </query>
          </union>
          <print mode="body"/>
          <recurse type="down"/>
          <print mode="meta"/>
        </osm-script>
        """
        output = run_overpass_query(query, exec_dir, db_dir)
        if output:
            pois = parse_pois_from_xml(output)
            all_pois.extend(pois)

    return all_pois


def add_amenities(map_object, coordinates, amenities, pruning_style):
    # Define the CSV file path to store amenities data specific to the pruning algorithm
    csv_file = f'ssudan/pruning/ssudan_amenities_data_{pruning_style}.csv'
    seen_pois = set()  # Set to keep track of processed POIs

    with open(csv_file, mode='a', newline='') as file:  # Use mode 'a' to append to the existing file
        writer = csv.writer(file)
        if os.stat(csv_file).st_size == 0:  # Check if file is empty to write the header
            writer.writerow(["amenity", "latitude", "longitude", "name"])  # Write header only if file is empty

        for route in coordinates:
            route_geometry = route['geometry']
            points = sample_route_points(route_geometry, num_samples=10)
            for amenity in amenities:
                pois = find_pois_along_route(points, amenity, search_radius=1000)
                for poi in pois:
                    poi_id = (poi['type'], poi['name'], poi['latitude'], poi['longitude'])
                    lat = float(poi.get('latitude'))
                    lon = float(poi.get('longitude'))
                    if poi_id not in seen_pois:
                        seen_pois.add(poi_id)  # Mark this POI as processed
                        folium.Marker(
                            location=(lat, lon),
                            icon=folium.CustomIcon(icon_image=modify_and_apply_svg('cross.svg'), icon_size=(12, 12)),
                            popup=f"{poi['name']} ({poi['type']})"
                        ).add_to(map_object)
                        writer.writerow([poi['type'], poi['latitude'], poi['longitude'], poi['name']])

    print(f"Amenities for {pruning_style} saved to {csv_file}")


def modify_and_apply_svg(svg_file_path):
    # Read SVG file
    with open(svg_file_path, 'r') as file:
        svg_data = file.read()

    # Define new colors
    colors = {
        'cross': '#FF0000',  # Red color for the circle
    }

    # Modify the SVG data by changing the fill attribute of specific paths
    paths = re.findall(r'<path[^>]*>', svg_data)
    if len(paths) > 0:
        svg_data = svg_data.replace(paths[0], re.sub(r'fill="[^"]*"', f'fill="{colors["cross"]}"', paths[0]))

    # Encode SVG data to base64
    svg_base64 = base64.b64encode(svg_data.encode('utf-8')).decode('utf-8')
    svg_url = f'data:image/svg+xml;base64,{svg_base64}'

    return svg_url


def write_routes_to_csv(routes_data, csv_file_path, pruning_style):
    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["name1", "name2", "distance"])
        for route in routes_data:
            name1 = route['name1']
            name2 = route['name2']
            distance = route['distance']
            if name1 != name2:  # Optional: check if names are the same
                writer.writerow([name1, name2, round(distance, 2)])
        print(f"Distances for {pruning_style} saved to {csv_file_path}")


def create_map_with_layers(locations_df, pruning_style='none'):
    # m = folium.Map(
    #     location=[locations_df.iloc[0]['latitude'], locations_df.iloc[0]['longitude']],
    #     tiles='https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png',
    #     attr='Map data © <a href="https://openstreetmap.org">OpenStreetMap</a> contributors',
    #     zoom_start=6
    # )

    m = folium.Map(
        location=[locations_df.iloc[0]['latitude']-2, locations_df.iloc[0]['longitude']],
        tiles='CartoDB Positron',  # Use the shorthand for CartoDB Positron
        attr='South Sudan Map',
        zoom_start=6
    )

    # HTML for the floating legend
    legend_html = '''
    <div style="position: fixed;
        top: 250px; right: 1000px; width: 160px; height: 160px;
        background-color: white; border:2px solid grey; z-index:9999; font-size:14px;
        padding: 10px;
        ">&nbsp; Location Graph <br>
        &nbsp; <i class="fa fa-circle fa-1x" style="color:green"></i>&nbsp; Camps <br>
        &nbsp; <i class="fa fa-circle fa-1x" style="color:yellow"></i>&nbsp; Towns <br>
        &nbsp; <i class="fa fa-circle fa-1x" style="color:red"></i>&nbsp; Conflict Zones <br>
        &nbsp; <i class="fa fa-plus fa-1x" style="color:red"></i>&nbsp; Health Facilities <br>
        &nbsp; <i class="fa fa-minus fa-1x" style="color:orange"></i>&nbsp; Routes <br>
    </div>
    '''

    # Create a floating legend as an HTML element
    legend_element = folium.Element(legend_html)

    # Add the legend to the map
    m.get_root().html.add_child(legend_element)

    # Add feature groups for the map
    extra_text_layer = FeatureGroup(name='Extra', show=True)
    location_text_layer = FeatureGroup(name='Text', show=True)
    location_layer = FeatureGroup(name='Locations', show=True)
    routes_layer = FeatureGroup(name='Routes', show=True)
    amenities_layer = FeatureGroup(name='Amenities', show=True)

    # Add layers to the map
    add_extra_text(extra_text_layer)
    add_location_texts(location_text_layer, locations_df)
    add_location_markers(location_layer, locations_df)

    # Conditional selection of the pruning algorithm
    if pruning_style == 'add_routes':
        routes_data = add_routes(m, locations_df)
    elif pruning_style == 'direct_distance':
        routes_data = add_direct_distance_routes(m, locations_df)
    elif pruning_style == 'visit_tracking':
        routes_data = add_visit_tracking_routes(m, locations_df)
    elif pruning_style == 'sequential':
        routes_data = add_sequential_pruned_routes(m, locations_df)
    elif pruning_style == 'triangle':
        routes_data = add_triangle_pruned_routes(m, locations_df, num_neighbors=3)
    else:
        raise ValueError("Invalid pruning style specified")

    # Write routes to the CSV
    csv_file_path = f'ssudan/pruning/ssudan_{pruning_style}_routes.csv'
    write_routes_to_csv(routes_data, csv_file_path, pruning_style)

    # Inquiry and add amenities to the map
    amenities = ['hospital']
    add_amenities(amenities_layer, routes_data, amenities, pruning_style)

    # Map layers order
    m.add_child(extra_text_layer, name='extra', index=0)
    m.add_child(routes_layer, name='routes', index=1)
    m.add_child(amenities_layer, name='amenities', index=2)
    m.add_child(location_layer, name='locations', index=3)
    m.add_child(location_text_layer, name='names', index=4)

    m.save(f'ssudan/pruning/ssudan_location_graph_{pruning_style}.html')
    print(f'Map for {pruning_style} saved to ssudan/pruning/ssudan_location_graph_{pruning_style}.html')


def colored(r, g, b, text):
    return f"\033[38;2;{r};{g};{b}m{text}\033[0m"


def main():
    # Set the file name of the locations.csv file
    locations_csv = f'{os.getcwd()}/ssudan/pruning/ssudan-small-locations.csv'
    # Load and prepare data
    df = pd.read_csv(locations_csv)

    # List of algorithms
    # algorithms = ['add_routes', 'direct_distance', 'visit_tracking', 'sequential', 'triangle']
    algorithms = ['add_routes']
    total_algorithms = len(algorithms)

    for i, alg in enumerate(algorithms):
        # Simulate running the algorithm
        create_map_with_layers(df, pruning_style=alg)

        # Calculate the percentage of completion
        percent_complete = ((i + 1) / total_algorithms) * 100

        # Print with color
        message = colored(0, 255, 255, f"Algorithm {alg} finalised successfully! {percent_complete:.2f}% completed.\n")
        sys.stdout.write("\r" + message)
        sys.stdout.flush()
        # Simulate some delay
        time.sleep(1)

    sys.stdout.write("\n" + colored(0, 255, 0, "All algorithms processed successfully.\n"))


if __name__ == "__main__":
    main()

In [ ]:
#### Create location graph manually

In [ ]:
def create_geojson(locations_df, routes_df):
    geojson_data = {
        "type": "FeatureCollection",
        "features": []
    }

    # Add locations as features
    for index, row in locations_df.iterrows():
        feature = {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [row['longitude'], row['latitude']]
            },
            "properties": {
                "name": row['name']
            }
        }
        geojson_data["features"].append(feature)

    # Add routes as features
    for index, row in routes_df.iterrows():
        source_filtered = locations_df.loc[locations_df['name'] == row['name1']]
        dest_filtered = locations_df.loc[locations_df['name'] == row['name2']]

        if not source_filtered.empty and not dest_filtered.empty:
            source = source_filtered.iloc[0]
            dest = dest_filtered.iloc[0]
            source_coords = [source['longitude'], source['latitude']]
            dest_coords = [dest['longitude'], dest['latitude']]

            # Request route from OSRM
            route_geometry = request_osrm_route(source_coords, dest_coords)
            if route_geometry:
                feature = {
                    "type": "Feature",
                    "geometry": {
                        "type": "LineString",
                        "coordinates": route_geometry
                    },
                    "properties": {
                        "distance": row['distance']
                    }
                }
                geojson_data["features"].append(feature)
        else:
            print(f"Missing location data for route from {row['name1']} to {row['name2']}")

    with open("./ssudan/pruning/output_map.geojson", "w") as geojson_file:
        json.dump(geojson_data, geojson_file, indent=4)


def request_osrm_route(source_coords, dest_coords):
    osrm_url = f"http://localhost:5000/route/v1/driving/{source_coords[0]},{source_coords[1]};{dest_coords[0]},{dest_coords[1]}?geometries=geojson&overview=full"
    response = requests.get(osrm_url)
    if response.status_code == 200:
        data = response.json()
        if data['code'] == 'Ok':
            return data['routes'][0]['geometry']['coordinates']
    return None


def create_html_map(locations_df, geojson_path):
    # Load the GeoJSON file into a GeoDataFrame
    gdf = gpd.read_file(geojson_path)

    # Merge the GeoDataFrame with the locations DataFrame to get the location_type
    gdf = gdf.merge(locations_df[['name', 'location_type']], on='name', how='left')

    # Initialize the folium map
    m = folium.Map(
        location=[locations_df.iloc[0]['latitude']-2, locations_df.iloc[0]['longitude']],
        tiles='CartoDB Positron',  # Use the shorthand for CartoDB Positron
        attr='South Sudan Map',
        zoom_start=6
    )

    # Handle lines (routes) separately
    routes = gdf[gdf.geometry.type == 'LineString']
    folium.GeoJson(routes,
                   name="Routes",
                   style_function=lambda x: {'color': 'orange', 'weight': 3}
                   ).add_to(m)

    # Handle points (locations) separately
    points = gdf[gdf.geometry.type == 'Point']
    for idx, row in points.iterrows():
        location_type = row['location_type']
        # Determine color based on location_type
        color = 'green' if location_type == 'camp' else 'yellow' if location_type == 'town' else 'red'

        # Add CircleMarker for each location
        folium.CircleMarker(
            location=[row.geometry.y, row.geometry.x],
            radius=6,
            color=color,
            weight=2,
            opacity=0.9,
            fill=True,
            fill_color=color,
            fill_opacity=0.6,
            popup=row['name']
        ).add_to(m)

        # Add text labels for each location
        icon = folium.DivIcon(html=f"""
            <div style="font-size:10pt; color: black;">{row['name']}</div>
            """)
        folium.Marker(
            location=[row.geometry.y, row.geometry.x],
            icon=icon,
        ).add_to(m)

    locations = [
        {'name': 'South Sudan', 'coordinates': [7.5, 32]},
        {'name': 'CAR', 'coordinates': [6.00, 24]},
        {'name': 'Ethiopia', 'coordinates': [8, 36]},
        {'name': 'Kenya', 'coordinates': [3, 35]},
        {'name': 'Sudan', 'coordinates': [15, 29]},
        {'name': 'Uganda', 'coordinates': [1, 32]},
        {'name': 'CRC', 'coordinates': [3, 29]},
    ]

    # Add markers with text labels
    for loc in locations:
        icon = folium.DivIcon(html=f"""
                <div style="font-size:12pt; color: black;">{loc['name']}</div>
                """)
        folium.Marker(
            location=loc['coordinates'],
            icon=icon
        ).add_to(m)

    # HTML for the floating legend
    legend_html = '''
    <div style="position: fixed;
        top: 250px; right: 1000px; width: 160px; height: 160px;
        background-color: white; border:2px solid grey; z-index:9999; font-size:14px;
        padding: 10px;
        ">&nbsp; Location Graph <br>
        &nbsp; <i class="fa fa-circle fa-1x" style="color:green"></i>&nbsp; Camps <br>
        &nbsp; <i class="fa fa-circle fa-1x" style="color:yellow"></i>&nbsp; Towns <br>
        &nbsp; <i class="fa fa-circle fa-1x" style="color:red"></i>&nbsp; Conflict Zones <br>
        &nbsp; <i class="fa fa-plus fa-1x" style="color:red"></i>&nbsp; Health Facilities <br>
        &nbsp; <i class="fa fa-minus fa-1x" style="color:orange"></i>&nbsp; Routes <br>
    </div>
    '''

    # Create a floating legend as an HTML element
    legend_element = folium.Element(legend_html)

    # Add the legend to the map
    m.get_root().html.add_child(legend_element)

    # Save the map to an HTML file
    m.save("./ssudan/pruning/output_map.html")
    return m


if __name__ == '__main__':
    locations_csv = './ssudan/pruning/locations.csv'
    routes_csv = './ssudan/pruning/routes.csv'

    locations_df = pd.read_csv(locations_csv)
    routes_df = pd.read_csv(routes_csv)

    create_geojson(locations_df, routes_df)

    map_object = create_html_map(locations_df, "./ssudan/pruning/output_map.geojson")
    print("Map has been created successfully.")